# Data cleaning
We will prepare the data for EDA. We will remove irrelevant features, impute missing values and remove extreme outliers

## 1. Import packages and data

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
filepath= "./jpn-hostel-data/raw.csv"
df = pd.read_csv(filepath)

In [3]:
df.head()

,Unnamed: 0,hostel.name,City,price.from,Distance,summary.score,rating.band,atmosphere,cleanliness,facilities,location.y,security,staff,valueformoney,lon,lat
0,1,"""Bike & Bed"" CharinCo Hostel",Osaka,3300,2.9km from city centre,9.2,Superb,8.9,9.4,9.3,8.9,9.0,9.4,9.4,135.513767,34.682678
1,2,& And Hostel,Fukuoka-City,2600,0.7km from city centre,9.5,Superb,9.4,9.7,9.5,9.7,9.2,9.7,9.5,NaN,NaN
2,3,&And Hostel Akihabara,Tokyo,3600,7.8km from city centre,8.7,Fabulous,8.0,7.0,9.0,8.0,10.0,10.0,9.0,139.777472,35.697447
3,4,&And Hostel Ueno,Tokyo,2600,8.7km from city centre,7.4,Very Good,8.0,7.5,7.5,7.5,7.0,8.0,6.5,139.783667,35.712716
4,5,&And Hostel-Asakusa North-,Tokyo,1500,10.5km from city centre,9.4,Superb,9.5,9.5,9.0,9.0,9.5,10.0,9.5,139.798371,35.727898


## 2. Tasks to do
- Remove irrelevant features
- Remove extreme outliers
- Impute missing data
- Make Distance a numerical feature

### 2.1 Remove irrelevant features

In [4]:
remove_features = ['Unnamed: 0', 'hostel.name', 'lon', 'lat']
df.drop(remove_features, axis=1, inplace=True)

In [5]:
df.shape

(342, 12)

### 2.2 Impute missing data

In [6]:
df.isna().sum()

City              0
price.from        0
Distance          0
summary.score    15
rating.band      15
atmosphere       15
cleanliness      15
facilities       15
location.y       15
security         15
staff            15
valueformoney    15
dtype: int64

In [7]:
numerical_features = ['summary.score', 'atmosphere', 'cleanliness', 
                      'facilities', 'location.y', 'security',
                      'staff', 'valueformoney']

# Strategy = median because of outliers
imputer = SimpleImputer(strategy='median')

# Fill missing values
df[numerical_features] = imputer.fit_transform(df[numerical_features])

In [8]:
categorical_features = ['rating.band']

imputer = SimpleImputer(strategy='most_frequent')

# Fill missing values
df[categorical_features] = imputer.fit_transform(df[categorical_features])

In [9]:
df.isna().sum()

City             0
price.from       0
Distance         0
summary.score    0
rating.band      0
atmosphere       0
cleanliness      0
facilities       0
location.y       0
security         0
staff            0
valueformoney    0
dtype: int64

### 2.3 Make Distance a numerical feature

In [10]:
df['Distance']

0       2.9km from city centre
1       0.7km from city centre
2       7.8km from city centre
3       8.7km from city centre
4      10.5km from city centre
                ...           
337     2.6km from city centre
338     2.9km from city centre
339    17.5km from city centre
340     2.4km from city centre
341     5.9km from city centre
Name: Distance, Length: 342, dtype: object

In [11]:
# Capture the number in the string
df['distance_km'] = df['Distance'].str.extract('(\d+(\.\d+)?)').astype(float)[0]
df['distance_km'].head()

0     2.9
1     0.7
2     7.8
3     8.7
4    10.5
Name: distance_km, dtype: float64

In [12]:
df['distance_km'].isna().sum()

0

In [13]:
df['distance_km'].describe().T

count    342.000000
mean       5.723099
std        4.590794
min        0.000000
25%        2.300000
50%        4.500000
75%        8.450000
max       36.600000
Name: distance_km, dtype: float64

In [14]:
# Check for nulls created
df[['distance_km','Distance']]

,distance_km,Distance
0,2.9,2.9km from city centre
1,0.7,0.7km from city centre
2,7.8,7.8km from city centre
3,8.7,8.7km from city centre
4,10.5,10.5km from city centre
...,...,...
337,2.6,2.6km from city centre
338,2.9,2.9km from city centre
339,17.5,17.5km from city centre
340,2.4,2.4km from city centre


In [15]:
df.drop(['Distance'], axis=1, inplace=True)

### 2.4 Remove extreme outliers
Remove outliers that do not make sense to exist.

In [16]:
df.shape

(342, 12)

In [17]:
def return_extreme_outliers(feature):
    # Calculate the IQR
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1

    # Define the upper and lower bounds 
    #(4.5 in order to detect outliers significantly far from the median )
    lower_bound = Q1 - 4.5 * IQR
    upper_bound = Q3 + 4.5 * IQR

    # Identify extreme outliers
    extreme_outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]

    # Print extreme outliers
    print("Extreme Outliers for '{feature}':")
    print(extreme_outliers[[feature]])
    return extreme_outliers[[feature]]

In [18]:
remove_lines = return_extreme_outliers('price.from').index
df.drop(remove_lines,inplace=True)

Extreme Outliers for '{feature}':
     price.from
289     1003200
300        7600
316     1003200


In [19]:
df.shape

(339, 12)

## 3. Save the new file
We will save the preprocessed data in order to be ready for the next step: EDA.

In [20]:
save_path = "./jpn-hostel-data/clean.csv"
df.to_csv(save_path, index=False)

## 4. Observation
Data shape was (342, 12) and now is (324, 12) meaning we lost 18 lines of data